# Desenvolvimento RAG

Anotações feitas pelo prof:

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

**Lista dos modelos:**

https://huggingface.co/models


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# Preparação do ambiente
Preparação do ambiente para execução do exemplo.

### Data e hora de execução

In [116]:
# Biblioteca de date
from datetime import datetime

data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M:%S')

print(data_e_hora_em_texto)

18/03/2024 14:50:09


### Tratamento de logs

Método para tratamento dos logs.

In [117]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Versão Python

In [118]:
# Biblioteca do sistema
import sys

print("Versão Python:", sys.version)

Versão Python: 3.11.8 (tags/v3.11.8:db85d51, Feb  6 2024, 22:03:32) [MSC v.1937 64 bit (AMD64)]


## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [119]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [120]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

Bibliota de dependência para manipular os embeddings pelo Langchain.

https://pypi.org/project/sentence-transformers/

Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

Dependências do xformers

https://pypi.org/project/lmdb/

Permite maior velocidade e menor consumo de memória nos transformers.

https://pypi.org/project/xformers/

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

https://huggingface.co/blog/4bit-transformers-bitsandbytes

https://pypi.org/project/bitsandbytes/

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

## Versão bibliotecas instaladas

## Carrega os documentos de PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

In [121]:
from tkinter import *
from tkinter import filedialog

def escolher_pdf():
    arquivo_pdf = filedialog.askopenfilename(title="Escolha o arquivo PDF")
    print(arquivo_pdf)
    return arquivo_pdf
pdf_escolhido = escolher_pdf()




C:/Users/miyun/Downloads/edital3_val.pdf


#### Carrega o PDF



In [122]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader


# Define o diretório
pdf = pdf_escolhido
print(pdf)
# Pode ser usado o PyPDFDirectoryLoader para um diretorio
carregador = PyPDFLoader(pdf)

# Carrega os documentos
documentos = carregador.load()

print(f"Documentos carregados: {len(documentos)}")

C:/Users/miyun/Downloads/edital3_val.pdf
Documentos carregados: 17


Exibe parte dos dados carregados do PDF ou HTML.

In [123]:
print("Quantidade de documentos(páginas):", len(documentos))
pagina = 2
print("Trecho página(", pagina, ") :", documentos[pagina].page_content[0:500])
print()
print("Metadados:", documentos[0].metadata)

Quantidade de documentos(páginas): 17
Trecho página( 2 ) :  
 
 3 de 17 
ferramentas de análise como Avaliação do Ciclo de Vida (ACV), Avaliação do Ciclo de 
Vida Social (ACVS) e a Economia Circular (EC). 
c.4 Pesquisa e desenvolvimento de tecnologias e processos para reuso e reciclagem 
de resíduos da produção de rochas ornamentais, agregados para construção civil, 
gesso, gemas e joias, cerâmica vermelha e cerâmica de revestimento, incluindo 
ferramentas de análise como Avaliação do Ciclo de Vida (ACV), Avaliação do Ciclo de 
Vida Social (ACVS) e a Ec

Metadados: {'source': 'C:/Users/miyun/Downloads/edital3_val.pdf', 'page': 0}


# 3 - Manda para a requisição GPT

In [124]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
prompt_sys_info_relevantes = """
    Você é um assistente que resume as informações mais relevantes para a inscrição de um edital. Você recebe como entrada o conteúdo do PDF do edital em questão e na saída você devolve as informações resumidas. Sua resposta deve ter no máximo 600 caractéres.
    Caso o trecho do PDF não tenha informações relevantes para inscrição, retorne "NULL".

    ##### EXEMPLO DE ENTRADA
    [conteúdo pdf do edital]

    ##### EXEMPLO DE SAÍDA
    [nome do edital],
    [datas de subimissão],
    [datas de fechamento de inscrição],
    [pré requisitos para inscrição],
    [maturidade da startup para submissão],
    [tipo de financiamento],
    ...

    """
prompt_sys_titulo = """ 
Você lê as duas primeiras páginas de um edital e extrai o título dele. Caso não encontre um título, crie um coerente ao conteúdo exposto. Retire todos os acentos e caractéres especiais do título. O título deve ter no máximo 60 caracteres.
    ##### EXEMPLO DE ENTRADA
    [conteúdo das 2 primeiras páginas do pdf do edital]

    ##### EXEMPLO DE SAÍDA
    [título sem acentos ou caracteres especiais]
"""
max_tokens = 400

key = os.getenv("OPENAI_API_KEY")

def chamada_gpt(prompt_user, prompt_sys):
    client = OpenAI(api_key=key)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": prompt_sys},
                  {"role": "user", "content": prompt_user}],
        temperature=1,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )

    print(response.choices[0].message.content)
    return response.choices[0].message.content


In [125]:
import tiktoken

def calcula_token(prompt, prompt_sys):
    dolar = 4.97
    modelo = "gpt-3.5-turbo"

    dict_modelo = {"gpt-4-0125-preview": 0.01, "gpt-4": 0.03, "gpt-4-32k": 0.06, "gpt-3.5-turbo-0125": 0.0005, "gpt-3.5-turbo-instruct": 0.0015, "gpt-3.5-turbo": 0.0015}


    codificador = tiktoken.encoding_for_model(modelo)
    lista_tokens_entrada = codificador.encode(prompt+prompt_sys)

    nro_tokens_entrada = len(lista_tokens_entrada)
    nro_tokens_saida = max_tokens
    nro_tokens_total = nro_tokens_saida + nro_tokens_entrada
    valor = dict_modelo[modelo]
    custo = float(nro_tokens_total/1000 * valor)

    print(f"Com um prompt de {nro_tokens_total} tokens o custo será de {custo} dolares ou {custo*dolar} reais")

    return custo



In [126]:
informacoes_relevantes = []

#Calculo de tokens
conteudo = []
quota_max = 0.1
quota = 0
titulo = "Não informado"

for pagina in documentos[:5]:
  quota += calcula_token(pagina.page_content, prompt_sys_info_relevantes)
for pagina in documentos[-5:]:
  quota += calcula_token(pagina.page_content, prompt_sys_info_relevantes)
quota += calcula_token(documentos[0].page_content + documentos[1].page_content, prompt_sys_titulo)
print(quota)

if quota < quota_max :
  for pagina in documentos[:5]:
    informacoes_relevantes.append(chamada_gpt(pagina.page_content, prompt_sys_info_relevantes))
  for pagina in documentos[-5:]:
    informacoes_relevantes.append(chamada_gpt(pagina.page_content, prompt_sys_info_relevantes))
  titulo = chamada_gpt(documentos[0].page_content + documentos[1].page_content, prompt_sys_titulo)

else:
  print("numero maximo de tokens ultrapassado")

Com um prompt de 1305 tokens o custo será de 0.0019575 dolares ou 0.009728775 reais
Com um prompt de 1443 tokens o custo será de 0.0021645 dolares ou 0.010757565 reais
Com um prompt de 1353 tokens o custo será de 0.0020295 dolares ou 0.010086615 reais
Com um prompt de 1356 tokens o custo será de 0.0020340000000000002 dolares ou 0.01010898 reais
Com um prompt de 1453 tokens o custo será de 0.0021795 dolares ou 0.010832115 reais
Com um prompt de 1450 tokens o custo será de 0.002175 dolares ou 0.010809749999999998 reais
Com um prompt de 1354 tokens o custo será de 0.0020310000000000003 dolares ou 0.01009407 reais
Com um prompt de 1263 tokens o custo será de 0.0018945 dolares ou 0.009415664999999998 reais
Com um prompt de 1260 tokens o custo será de 0.00189 dolares ou 0.009393299999999999 reais
Com um prompt de 1205 tokens o custo será de 0.0018075 dolares ou 0.008983275 reais
Com um prompt de 2105 tokens o custo será de 0.0031575 dolares ou 0.015692775 reais
0.0233205


2024-03-18 14:50:18,622 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


SELEÇÃO PÚBLICA MCTI/FINEP/FNDCT - Subvenção Econômica à Inovação – 10/2022, 
Datas de submissão: Não especificadas, 
Datas de fechamento de inscrição: Não especificadas, 
Pré-requisitos para inscrição: Desenvolver produtos, processos e/ou serviços inovadores relacionados às linhas temáticas, 
Maturidade da startup para submissão: Projetos de risco tecnológico de níveis de maturidade tecnológica (TRLs) 4 a 7, 
Tipo de financiamento: Recursos de subvenção econômica.


2024-03-18 14:50:19,888 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


NULL.


...

Seleção Pública Finep
Datas: 
- Recurso: 03/11/2022
- Resultado Final: 14/11/2022
Pré-requisitos: Conformidade com itens do edital e legislação vigente. Compromisso com veracidade das informações.
Maturidade da startup: Nível de maturidade tecnológica definido em anexo.
Outras informações: Acompanhamento técnico e financeiro pela Finep. Proteção da propriedade intelectual obrigatória. Possibilidade de revogação ou anulação do processo seletivo.
NULL


2024-03-18 14:50:33,701 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Seleção Pública Finep:
- Não há datas de submissão ou fechamento de inscrição mencionadas.
- Pré requisitos: Consentimento dos titulares dos dados pessoais e documentos específicos.
- Maturidade da startup: Não especificada.
- Tipo de financiamento: Subvenção econômica.
- Contato para dúvidas: sac@finep.gov.br.
- Penalidades para descumprimento das normas e legislação vigente.
- Omissões serão decididas pelo Diretor da Diretoria de Inovação.


2024-03-18 14:50:35,248 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


SELECAO PUBLICA MCTI FINEP FNDCT Subvencao Economica a Inovacao 10/2022 Programa Mineracao e Desenvolvimento


In [127]:
prompt_verifica = """
Você é um sistema que classifica a maturidade a qual uma startup precisa apresentar para a subimissão do edital. Como entrada você recebe informações relevantes do edital. Na saída é apenas UMA das MATURIDADES POSSÍVEIS

#### MATURIDADES POSSÍVEIS
Ideacao: Empreendedor potencial trabalhando em uma ideia de negócio. É o estágio inicial.
Validacao: Uma startup iterando em direção ao encaixe entre Problema e Solução, validando MVPs enquanto busca clientes pagantes.
Operacao: Empresa com um portfólio estável de clientes pagantes, consistentemente melhorando sua operação em direção à previsibilidade de receita e o encaixe entre Produto e Mercado.

### EXEMPLO DE ENTRADA
[informações relevantes do edital]

### EXEMPLO DE SAÍDA
Ideacao

"""

In [128]:
respostas = []
for i in informacoes_relevantes:
  if i != "NULL":
    respostas.append(chamada_gpt(i, prompt_verifica))
  else:
    informacoes_relevantes.remove("NULL")

2024-03-18 14:50:36,598 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Operacao


2024-03-18 14:50:38,370 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Por favor, forneça informações relevantes do edital para que eu possa determinar a maturidade necessária para a submissão.


2024-03-18 14:50:40,070 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Operacao


2024-03-18 14:50:41,739 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Validacao


2024-03-18 14:50:42,571 : INFO : HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Ideacao


In [129]:
respostas_validas = []
for j in respostas:
  if j == "Operacao" or j == "Ideacao" or j == "Validacao":
    respostas_validas.append(j)

In [130]:
from collections import Counter

def termo_mais_frequente(lista_strings):
    # Concatenando todas as strings em uma única string
    todas_as_palavras = ' '.join(lista_strings)
    # Dividindo a string em palavras individuais
    palavras = todas_as_palavras.split()
    # Contando a ocorrência de cada palavra
    contagem = Counter(palavras)
    # Encontrando o termo mais frequente
    termo_mais_frequente, frequencia = contagem.most_common(1)[0]

    return termo_mais_frequente

In [131]:
etapa = termo_mais_frequente(respostas_validas)
print(titulo, etapa)

SELECAO PUBLICA MCTI FINEP FNDCT Subvencao Economica a Inovacao 10/2022 Programa Mineracao e Desenvolvimento Operacao


In [134]:
import csv
# Abrir o arquivo CSV em modo de escrita e crie um escritor CSV
with open("resultados.csv", mode='a', newline='') as arquivo_csv:
    escritor_csv = csv.writer(arquivo_csv)
    # Escreva os dados nas linhas do CSV
    escritor_csv.writerow([titulo, etapa])